In [4]:
# !curl -O https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
# !curl -O https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3832  100  3832    0     0  19598      0 --:--:-- --:--:-- --:--:-- 19752


In [9]:
#!pip install tqdm notebook==7.1.2 openai elasticsearch pandas scikit-learn ipywidgets

In [ ]:
# !pip install python-dotenv

In [ ]:
#!pip install requests tiktoken

In [1]:
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
from dotenv import load_dotenv
import openai
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)


In [9]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [12]:
index.fit(documents)

In [18]:
q = 'cuantos idiomas entiendes?'

In [13]:
#client = OpenAI()
client = openai.OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [19]:
response = client.chat.completions.create(
    model='phi3',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

' Como asistente de IA, puedo procesar y generar texto en múltiples idiomas incluyendo inglés, español, francés, chino simplificado, alemán, entre otros. No "entiendo" idiomas como lo haría un humano, pero puedo manejarlos mediante programación e inteligencia artificial.'

In [20]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
        #model='moondream',
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [24]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
rag(query)

" To run Kafka in the terminal for a Java producer or consumer, you will need to execute your Java file specifying its classpath with JAR files necessary for Kafka. For instance:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your Kafka JAR file, and adjust paths as needed for your project's structure."

In [26]:
rag('the course has already started, can I still enroll?')

" Yes, even if you don't register before the course starts, you're still eligible to submit homeworks after the start date. However, there will be deadlines for turning in final projects, so it's advised not to leave everything for the last minute.\n\nAdditionally, once the course is finished, all materials and resources will remain available, allowing you to follow the course at your own pace. You can also continue working on homeworks or prepare for future cohorts."

In [27]:
from elasticsearch import Elasticsearch

In [28]:
es_client = Elasticsearch('http://localhost:9200') 

In [29]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [30]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [31]:
from tqdm.auto import tqdm

In [32]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [33]:
query = 'I just disovered the course. Can I still join it?'

In [34]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [35]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [36]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [37]:
rag(query)

" Yes, you can still join the course even if you discover it after the start date. The materials will be available for you to follow at your own pace, and you'll be able to submit homeworks and prepare for final projects as needed. However, please keep in mind that there are deadlines for turning in final projects, so starting early is recommended."

In [ ]:
pip install requests